In [374]:
import tkinter as tk
#from PIL import ImageTk, Image
from requests import request
#from io import BytesIO
from IPython.display import Image, HTML
#from IPython.core.display import HTML 
import math
from functools import reduce

In [375]:
# Initial Dealer Setup
class Dealer:
    def __init__(self):
        self.name = 'Dealer'
        self.cards = []
        self.hidden_card = "https://deckofcardsapi.com/static/img/XX.png"
        self.totals = []
        self.status = 'Active'
        self.final = 0
    
    def add_card(self, card):
        return self.cards.append(card)
    
    def clear_cards(self):
        self.cards = []
        return self.cards
            

In [376]:
# Initial Player Setup

class Player(Dealer):
    def __init__(self):
        self.name = self.set_name()
        self.balance = 1000
        self.cards = []
        self.bets = []
        self.status = []
        self.veredict = []
        self.totals = []
    
    def set_name(self):
        name = input('Ingresa tu nombre: ')
        return name
        
    def bet(self):
        print(f"Tu balance actual es de {self.balance}. Ingresa tu apuesta, recuerda que tiene que ser un número entero")
        while True:    
            try:
                bet = int(input('Apuesta: '))
                if bet <= self.balance:
                    self.bets.append(bet)
                    self.balance -= bet
                    break
                else:
                    print("No puedes apostar más de lo que tienes en tu balance. Intenta una apuesta menor.")
    
            except ValueError:
                print("No ingresaste un número intentalo de nuevo.")
                
            
    def add_card(self, card):
        self.cards.append(card)
    
    def clear_cards(self):
        self.cards = []
        return self.cards

In [401]:
#Initial game Setup

class Game:
    def __init__(self):
        self.player = Player()
        self.dealer = Dealer()
        self.deck = self.initialize_deck()
        self.cards_remaining = 0
        self.turn = 'player'
        self.deal_cards()
        self.print_game()
        
    def initialize_deck(self):
        cards = 'https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=6'
        response = request('GET',cards).json()
        return response['deck_id']

    def draw(self):
        draw = f'https://deckofcardsapi.com/api/deck/{self.deck}/draw/?count=1'
        drawn_card = request('GET', draw).json()
        img = drawn_card['cards'][0]['images']['png']
        value = drawn_card['cards'][0]['value']
        code = drawn_card['cards'][0]['code']
        self.cards_remaining = drawn_card['remaining']
        return [img, value, code]

    def deal_cards(self):
        self.player.cards = []
        self.player.bets = []
        self.dealer.cards = []
        self.player.status = []
        self.player.totals = []
        self.player.veredict = []
        self.dealer.totals = []
        self.dealer.status = 'Active'
        self.dealer.final = 0
        self.turn = 'player'
        if self.player.balance > 0:
            self.player.bet()
            self.player.cards.append([self.draw()])
            self.dealer.cards.append([self.draw()])
            self.player.cards[0].append(self.draw())
            self.dealer.cards[0].append(self.draw())
            self.player.status.append('Active')
            self.player.totals.append([])
            self.dealer.totals.append([])
            self.print_game()
            return self.initial_bj_evaluation()
        else:
            resp = input('Ya no tienes balance, deseas comenzar un juego nuevo? (s) Sí, (n) No')
            if resp == 's':
                game.__init__()
    
    def hand_total(self, who):
        for i in range(0,len(who.cards)):
            hand = who.cards[i]
            print(hand)
            hand_values = [card[1] for card in hand]
            print(hand_values)
            total1 = 0
            total2 = 0
            if len(hand_values) == 2 and 'ACE' in hand_values and ('KING' in hand_values or 'QUEEN' in hand_values or 'Jack' in hand_values or '10' in hand_values):
                who.totals[i] = '21 - BLACKJACK!'
                return who.totals[i]
            for value in hand_values:
                if value == 'ACE':
                    total1 += 1
                    total2 += 11
                elif value == 'KING' or value == 'QUEEN' or value == 'JACK':
                    total1 += 10
                    total2 += 10
                else:
                    total1 += int(value)
                    total2 += int(value)
            who.totals[i] = [total1, total2]
            return who.totals[i]
                
    def initial_bj_evaluation(self):
        self.hand_total(self.player)
        self.hand_total(self.dealer)
        if '21 - BLACKJACK!' in self.player.totals:
            self.player.balance += self.player.bets[0] + math.floor(self.player.bets[0]*3/2)
            self.player.veredict[0] = "BLACKJACK! YOU WIN! 3:2"
            self.player.status[0] = 'Inactive'
            return self.deal_cards()
        elif '21 - BLACKJACK' in self.dealer.totals:
            self.turn = 'Dealer'
            self.player.veredict[0] = "DEALER BLACKJACK - YOU LOSE!"
            self.player.status[0] = 'Inactive'
            self.print_game()
            return self.deal_cards()
        else:
            return self.player_action()
    
    def player_action(self):
        for i in range(len(self.player.cards)):
            while self.player.status[i] == 'Active':
                if len(self.player.cards[i]) == 2 and self.player.cards[i][0][1] == self.player.cards[i][1][1] and self.player.balance > self.player.bets[i]:
                    action = input('Ingresa tu acción - (s) Split, (d) Double Down, (n) Hit - New Card, (h) Hold or Stand\n')
                    if action == 's':
                        self.split(i)
                        self.print_game()
                    if action == 'd':
                        self.double_down(i)
                        self.print_game()
                    if action == 'n':
                        self.new_card(i)
                        self.print_game()
                    if action == 'h':
                        self.player.status[i] = 'Inactive'
                        self.player.veredict.insert(i, "N/A")
                        self.print_game()
                elif len(self.player.cards[i]) == 2 and self.player.balance > self.player.bets[i]:
                    action = input('Ingresa tu acción - (d) Double Down, (n) Hit - New Card, (h) Hold or Stand\n')
                    if action == 'd':
                        self.double_down(i)
                        self.print_game()
                    if action == 'n':
                        self.new_card(i)
                        self.print_game()
                    if action == 'h':
                        self.player.status[i] = 'Inactive'
                        self.player.veredict.insert(i, "N/A")
                        self.print_game()
                else:
                    action = input('Ingresa tu acción - (n) Hit - New Card, (h) Hold or Stand\n')
                    if action == 'n':
                        self.new_card(i)
                        self.print_game()
                    if action == 'h':
                        self.player.status[i] = 'Inactive'
                        self.player.veredict.insert(i, "N/A")
                        self.print_game()
        self.turn = 'Dealer'
        self.print_game()
        return self.dealer_action()

    
    def dealer_action(self):
        while self.dealer.status == 'Active':
            self.hand_total(self.dealer)
            if int(self.dealer.totals[0][0]) > 16 and int(self.dealer.totals[0][1]) > 16:
                self.dealer.status = 'Inactive'
                self.dealer.final = max(self.dealer.totals[0][0], self.dealer.totals[0][1])
            else:
                self.dealer.cards[0].append(self.draw())
        return self.final_evaluation()
            
    def final_evaluation(self):
        for cards in self.player.cards:
            i = self.player.cards.index(cards)
            if self.player.veredict[i] == 'N/A':
                player_final = max(int(self.player.totals[i][0]), int(self.player.totals[i][0]))
                if player_final > int(self.dealer.final) or int(self.dealer.final) > 21:
                    self.player.balance += self.player.bets[i]*2
                    self.player.veredict[i] = 'Win'
                if player_final == int(self.dealer.final):
                    self.player.balance += self.player.bets[i]
                    self.player.veredict[i] = "Push"
                if player_final < int(self.dealer.final):
                    self.player.veredict[i] = "Lose"
        return self.deal_cards()
                    
    def split(self, index):
        self.player.bets.insert(index, self.player.bets[index])
        self.player.balance -= self.player.bets[index]
        original1 = self.player.cards[index][0]
        original2 = self.player.cards[index][1]
        self.player.cards[index] = [original1, self.draw()]
        self.player.cards.append([original2, self.draw()])
        self.player.status.append('Active')
        self.player.totals.append([])
        self.hand_total(self.player)
        if self.player.totals[index] == '21 - BLACKJACK!':
            self.player.balance += self.player.bets[index] + math.floor(self.player.bets[index]*3/2)
            self.player.status[index] = 'Inactive'
            self.player.veredict.insert(index, "BLACKJACK! YOU WIN! 3:2")
        if self.player.totals[index +1] == '21 - BLACKJACK!':
            self.player.balance += self.player.bets[index+1] + math.floor(self.player.bets[index+1]*3/2)
            self.player.status[index +1] = 'Inactive'
            self.player.veredict.insert(index +1, "BLACKJACK! YOU WIN! 3:2")
        return self.player_action
        
        
        
    def double_down(self, index):
        self.player.balance -= self.player.bets[index]
        self.player.bets[index] += self.player.bets[index]
        hand = self.player.cards[index].append(self.draw())
        self.hand_total(self.player)
        self.player.status[index] = 'Inactive'
        if self.player.totals[index][0] > 21 or self.player.totals[index][1] > 21:
            self.player.veredict.insert(index, 'Bust! - YOU LOSE')
        else:
            self.player.veredict.insert(index, 'N/A')
        return self.player_action()
                
    def new_card(self, index):
        self.player.cards[index].append(self.draw())
        self.hand_total(self.player)
        if self.player.totals[index][0] > 21 and self.player.totals[index][1] > 21:
            self.player.status[index] = 'Inactvie'
            self.player.veredict.insert(index, 'Bust! - YOU LOSE')
        return self.player_action()
        
    
    def print_game(self):
        dealer_cards = ""
        player_cards = ""
        if self.turn == "player":
            dealer_cards = f"""
            <td><img src={self.dealer.hidden_card} style="width: 60px;"/></td>
            <td><img src={self.dealer.cards[0][1][0]} style="width: 60px;"/></td>
            """
        else:
            for card in self.dealer.cards[0]:
                dealer_cards += f"""<td><img src={card[0]} style="width: 60px;"/></td>"""
        
        for card in self.player.cards[0]:
            player_cards += f"""<td><img src={card[0]} style="width: 60px;"/></td>"""
        
        return HTML(f""" 
                <table>
                <tr><h1> Dealer Hand </h1></tr>
                <tr>
                {dealer_cards}
                </tr>
                </table>
                
                <table>
                <tr><h1> {self.player.name}'s Hand </h1></tr>
                <tr>
                {player_cards}
                </tr>
                </table>
        """)

In [399]:
#Begin Game
game = Game()

Ingresa tu nombre: Rob
